In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jobathonfebruary2022/test_1zqHu22.csv
/kaggle/input/jobathonfebruary2022/sample_submission_JPlpRcN.csv
/kaggle/input/jobathonfebruary2022/train_0OECtn8.csv


In [2]:
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

In [3]:
!pip install pycaret

     |████████████████████████████████| 301 kB 480 kB/s            
     |████████████████████████████████| 114 kB 6.8 MB/s            
  Preparing metadata (setup.py) ... - done
     |████████████████████████████████| 15.6 MB 6.6 MB/s            
     |████████████████████████████████| 167 kB 71.8 MB/s            
     |████████████████████████████████| 10.4 MB 20.2 MB/s            
     |████████████████████████████████| 24.2 MB 528 kB/s             
     |████████████████████████████████| 636 kB 59.3 MB/s            
     |████████████████████████████████| 25.9 MB 4.7 kB/s             
     |████████████████████████████████| 184 kB 63.5 MB/s            
     |████████████████████████████████| 1.0 MB 69.0 MB/s            
     |████████████████████████████████| 14.8 MB 61.4 MB/s            
     |████████████████████████████████| 79 kB 5.7 MB/s             
     |████████████████████████████████| 58 kB 4.1 MB/s             
  Preparing metadata (setup.py) ... - done
  Created whe

In [4]:
#import modules
import matplotlib.pyplot as plt # for plotting graphs
import seaborn as sns # for plotting graphs
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

In [5]:
train = pd.read_csv('../input/jobathonfebruary2022/train_0OECtn8.csv')
test = pd.read_csv('../input/jobathonfebruary2022/test_1zqHu22.csv')
subm = pd.read_csv('../input/jobathonfebruary2022/sample_submission_JPlpRcN.csv')

In [6]:
print(train.shape)
print(test.shape)

(89197, 10)
(11121, 9)


In [7]:
def preproc(train,test):
    label_encoder = LabelEncoder()

    train['gender']= label_encoder.fit_transform(train['gender'])
    test['gender']= label_encoder.transform(test['gender'])
    
    #ordinal encoding of profession
    prof_map = {'Other':0,'Student':1,'Working Professional':2}
    train['profession'] = train['profession'].map(prof_map)
    test['profession'] = test['profession'].map(prof_map)

In [8]:
preproc(train,test)

In [9]:
targetcol = 'engagement_score'
target = train[targetcol]

In [10]:
exclude_cols =['row_id','user_id']
features =[col for col in train.columns if col not in exclude_cols]
print(len(features))
print(features)
features_excl_target = [col for col in features if col!=targetcol]

8
['category_id', 'video_id', 'age', 'gender', 'profession', 'followers', 'views', 'engagement_score']


In [11]:
# catcols = ['user_id','category_id','video_id','gender']
catcols = ['category_id','video_id','gender']
numeric_cols = [col for col in features if col not in catcols+[targetcol]]
print(numeric_cols)

['age', 'profession', 'followers', 'views']


In [12]:
from pycaret.regression import *

In [13]:
s = setup(train[features], session_id=100,
          silent=True,
          numeric_features=numeric_cols,
          categorical_features=catcols,
#           high_cardinality_features=['user_id'],
#           high_cardinality_method='frequency',
          target = targetcol
#           preprocess=False,
#           use_gpu=True,
#           normalize=True,# normalize_method='minmax',
          )

,Description,Value
0,session_id,100
1,Target,engagement_score
2,Original Data,"(89197, 8)"
3,Missing Values,False
4,Numeric Features,4
5,Categorical Features,3
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(62437, 217)"


In [14]:
X_train = get_config('X_train')
print(len(X_train.columns))
X_train.columns

217


Index(['age', 'profession', 'followers', 'views', 'category_id_1.0',
       'category_id_10.0', 'category_id_11.0', 'category_id_12.0',
       'category_id_13.0', 'category_id_15.0',
       ...
       'video_id_91.0', 'video_id_92.0', 'video_id_93.0', 'video_id_94.0',
       'video_id_95.0', 'video_id_96.0', 'video_id_97.0', 'video_id_98.0',
       'video_id_99.0', 'gender_0'],
      dtype='object', length=217)

In [15]:
%%time
# lgbm = create_model('lightgbm',early_stopping_rounds=100,eval_metric='auc',
#                     num_boost_round=1000)
lgbm = create_model('lightgbm')
lgbm

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.5222,0.4917,0.7012,0.3693,0.1975,0.2480
1,0.5199,0.4732,0.6879,0.3585,0.1869,0.2459
2,0.5338,0.5097,0.7139,0.3389,0.1970,0.2965
3,0.5162,0.4702,0.6857,0.3633,0.1868,0.2256
4,0.5247,0.4854,0.6967,0.3598,0.1915,0.2065
5,0.5276,0.4888,0.6992,0.3444,0.1891,0.2330
6,0.5177,0.4718,0.6868,0.3540,0.1846,0.2000
7,0.5311,0.4932,0.7023,0.3446,0.1919,0.2413
8,0.5135,0.4566,0.6757,0.3692,0.1817,0.2451
9,0.5230,0.4766,0.6904,0.3695,0.1899,0.2690


CPU times: user 2.08 s, sys: 224 ms, total: 2.31 s
Wall time: 10.5 s


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=100, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [16]:
%%time
predict_model(lgbm)
final_model = finalize_model(lgbm)
test_preds = predict_model(final_model, data=test)
print(test_preds.head())

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,0.5448,0.5120,0.7155,0.3051,0.1953,0.2695


   row_id  user_id  category_id  video_id  age  gender  profession  followers  \
0   89198     7986           12        42   14       1           1        180   
1   89199    11278           34       115   14       1           1        230   
2   89200    17245            8       110   44       0           2        280   
3   89201     9851           16       137   18       1           1        270   
4   89202    16008           34        96   47       0           0        230   

   views     Label  
0    138  4.092684  
1    840  3.758773  
2    628  2.685152  
3    462  3.891776  
4    840  2.181195  
CPU times: user 2.9 s, sys: 146 ms, total: 3.04 s
Wall time: 4.98 s


In [17]:
submission = test_preds[['row_id','Label']]
submission=submission.rename(columns={'Label':'engagement_score'})
submission[['row_id','engagement_score']].to_csv('submission.csv',index=False)

In [18]:
test_sub = pd.read_csv('submission.csv')

In [19]:
print(test_sub.shape)
test_sub.head()

(11121, 2)


,row_id,engagement_score
0,89198,4.092684
1,89199,3.758773
2,89200,2.685152
3,89201,3.891776
4,89202,2.181195
